In [1]:
import os 
from tqdm import tqdm, trange
import torch 
import torch.nn as nn
import numpy as np
import torch.nn.functional as F

In [6]:
data_path = "./data/names.txt"

with open(data_path, "r") as f:
    raw_names = f.readlines()

names_processed  = [name.lower().strip() for name in raw_names]

In [13]:
def get_vocab(list_of_names):
    vocab = {}
    idx = 0
    for each in list_of_names:
        for char in each:
            if char not in vocab:
                vocab[char] = idx
                idx+=1
    vocab[idx] = "*"
    idx+=1
    vocab[idx] = "<EOF>"
    return vocab